In [ ]:
!pip install tensorflow-gpu

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
#resize all images 
IMAGE_SIZE = [224,224]

train_path = '/content/drive/MyDrive/tomatoset/train'
valid_path = '/content/drive/MyDrive/tomatoset/valid'

In [ ]:
# Inception v3
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# ignoring training from existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# getting the labels
folders = glob('/content/drive/MyDrive/tomatoset/train/*')
folders

In [ ]:
# layers
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)


In [ ]:
# creating  model
model = Model(inputs=inception.input, outputs=prediction)

# structure model
model.summary()

In [ ]:
#cost and optimization
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/tomatoset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/tomatoset/valid',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

In [ ]:
#train model 
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
from tensorflow.keras.models import load_model

model.save('detector.tflite')